In [93]:
SUITS = ["♠", "♥", "♦", "♣"]
RANKS = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "10", "J", "Q", "K"]
VALUES = [11, 2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10]

In [94]:
class Card:
    def __init__(self, rank, suit):
        self.suit = suit
        self.rank = rank
        self.value = VALUES[RANKS.index(rank)]
    
    def __int__(self):
        return self.value
    
    def __str__(self):
        return self.rank + self.suit
    
    def __repr__(self):
        return f"<CARD: {self.rank}{self.suit} ({self.value})>"

In [95]:
class Hand:
    def __init__(self, cards=None):
        if cards is None:
            self.cards = []
        else:
            assert all([isinstance(card, Card) for card in cards])
            self.cards = cards
        self.evaluate()
        
    
    def evaluate(self):
        self.value = sum([card.value for card in self.cards])
        self.aces = [card.rank for card in self.cards].count('A')
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
    
    def add_card(self, card):
        self.cards.append(card)
        self.evaluate()
        return self.value
    
    def get_score(self):
        if self.value > 21:
            return 0
        elif self.is_blackjack():
            return 22
        else:
            return self.value

    def get_lowest_value(self):
        return self.value - ( 10 * self.aces )
    
    def is_blackjack(self):
        return self.value == 21 and len(self.cards) == 2

    def __repr__(self):
        return f'<Hand: {self.value} {[str(card) for card in self.cards]}>'

In [96]:
class Deck():
    def __init__(self, number_decks=6):
        self.cards = [Card(a[0],a[1]) for a in np.transpose([np.repeat(RANKS, len(SUITS)), np.tile(SUITS, len(RANKS))])] * number_decks
    
    def shuffle(self):
        np.random.shuffle(self.cards)
    
    def deal(self):
        return self.cards.pop(0)
    
    def __str__(self):
        return str(self.cards)
    
    def __len__(self):
        return len(self.cards)
    
    def get_rank_counts(self):
        return pd.Series([card.rank for card in self.cards]).value_counts()

    def get_value_counts(self):
        return pd.Series([card.value for card in self.cards]).value_counts().sort_index()

    def get_chance_of_value(self, value):
        if value == 1:
            return self.get_rank_counts().loc["A"]/len(self.cards)
        return self.get_value_counts().loc[value]/len(self.cards)
    
    def get_chance_of_rank(self, rank):
        return self.get_rank_counts().loc[rank]/len(self.cards)
    
    def __repr__(self):
        return f"<Deck: {len(self.cards)} cards>"
    
    def chance_of_blackjack(self):
        return self.get_chance_of_rank("A") * self.get_chance_of_value(10) * 2



In [97]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import matplotlib.pyplot as plt
import time


In [111]:
class TempHand:
    def __init__(self, chance, score, aces, dist):
        self.chance = chance
        self.score = score
        self.aces = aces
        self.dist = dist
    
    def get_possible_next(self):
        possible = []
        for val in self.dist.index:
            if self.dist[val] > 0:
                newchance = self.chance*(self.dist.loc[val]/self.dist.sum())
                # print("newchance", val)
                # print(self.dist.loc[val])
                # print(self.dist.sum())
                newscore = self.score + val
                newaces = self.aces
                if val == 11:
                    newaces += 1
                if newscore > 21 and newaces > 0:
                    newscore -= 10
                    newaces -= 1
                newdist = self.dist.copy()
                newdist[val] -= 1
                possible.append(TempHand(newchance, newscore, newaces, newdist))
        return possible
    
    def __repr__(self):
        return str((self.chance, self.score, self.aces))

In [99]:
def get_stand_chances(player, dealer, deck):
    probs = get_dealer_score_probabilities(deck, dealer)
    push = 0.0 if player.get_score() < 17 else probs[player.get_score()]
    lose = probs[probs.index > player.get_score()].sum()
    win = probs[probs.index < player.get_score()].sum()
    # print(push + lose + win)
    return (win, push, lose)

In [118]:
# deck = Deck()
# card1 = deck.deal()
# card2 = deck.deal()
# dealer=Hand(cards=[card1, card2])
# print(dealer)
# print(dealer.aces)
# print(dealer.get_score())

<Hand: 12 ['A♠', 'A♥']>
1
12


In [109]:
possible_values = [2,3,4,5,6,7,8,9,10,11]
possible_values_arr = ["num2","num3","num4","num5","num6","num7","num8","num9","num10","num11"]

def get_dealer_score_probabilities(deck, dealer = None):
    # print("cards in deck", len(deck.cards))
    vals = deck.get_value_counts()
    # print(vals/vals.sum())

    cards_dealt = 0
    
    if dealer != None:
        firstrow = [1, dealer.get_score(), dealer.aces] + vals.values.tolist()
        cards_dealt += len(dealer.cards)
    else:
        firstrow = [1, 0, 0] + vals.values.tolist()

    possible = pd.DataFrame([firstrow], columns=["chance", "score", "aces"] + possible_values_arr)

    final = pd.DataFrame([], columns=possible.columns)
    probability_of_blackjack = 0.0

    while len(possible) > 0:
        cards_dealt += 1

        next_round_possibles = pd.DataFrame([], columns=possible.columns)
        for possible_value in possible_values:
            
            new_possibles = possible.copy()
            new_possibles["chance"] *= (new_possibles[f"num{possible_value}"]/new_possibles[possible_values_arr].sum(axis=1))
            # print(new_possibles[f"num{possible_value}"])
            # print(new_possibles[possible_values_arr].sum(axis=1))
            # print(new_possibles["chance"])
            new_possibles["score"] += possible_value
            new_possibles["aces"] += (possible_value == 11)
            acepossible = (new_possibles["score"] > 21) & (new_possibles["aces"] > 0)
            new_possibles.loc[acepossible,"score"] -= 10
            new_possibles.loc[acepossible,"aces"] -= 1
            new_possibles[f"num{possible_value}"] -= 1
            next_round_possibles = pd.concat((next_round_possibles, new_possibles), axis=0, ignore_index=True)
        
        final = pd.concat((final, next_round_possibles[next_round_possibles["score"] >= 17]), axis=0, ignore_index=True)
        possible = next_round_possibles[next_round_possibles["score"] < 17]

        if cards_dealt == 2:
            # print("round 2 final")
            # print(final)
            # print("round 2 blackjack")
            # print(final[final["score"] == 21])
            probability_of_blackjack = sum(final[final["score"] == 21]["chance"])
    
    # print(final)
    scores = final[["score","chance"]].values
    valcounts = pd.DataFrame(scores).groupby(0).sum()
    valcounts = pd.Series(index=valcounts.index.values.astype(int), data=valcounts.values.flatten())
    valcounts[0] = valcounts[valcounts.index > 21].sum()
    valcounts = valcounts[valcounts.index <= 21].sort_index(ascending=True)
    valcounts[21] -= probability_of_blackjack
    valcounts[22] = probability_of_blackjack

    # TO NORM ALL EXCEPT DEALER BLACKJACK, USE:
    # valcounts[:21] /= (1 - probability_of_blackjack)

    # print(f"\n{valcounts}")  
    # print("SUM", valcounts.sum())
    return valcounts


# valcounts = get_dealer_score_probabilities_opt(deck, dealer=dealer)

cards in deck 310

0     0.244856
17    0.150225
18    0.150971
19    0.151401
20    0.151063
21    0.151484
22    0.000000
dtype: float64
SUM 1.0000000000000009


In [122]:
def get_dealer_score_probabilities_old(deck, dealer = None):
    print("cards in deck", len(deck.cards))
    score = np.zeros(28)
    vals = deck.get_value_counts()
    # print(vals/vals.sum())

    cards_dealt = 0
    possible = [TempHand(1.0, 0, 0, vals)]

    if dealer != None:
        cards_dealt += len(dealer.cards)
        possible = [TempHand(1.0, dealer.get_score(), dealer.aces, vals)]
    # print(possible)
    final = []
    
    probability_of_blackjack = 0.0

    while len(possible) > 0:
        cards_dealt += 1
        # print(cards_dealt, end=" ")
        for temp in possible:
            # print("analyzing", temp)
            newpossible = temp.get_possible_next()
            # print("new possible", newpossible)
            possible = possible + newpossible
            possible.remove(temp)
        newfinal = [temp for temp in possible if temp.score >= 17]
        final = final + newfinal
        possible = [temp for temp in possible if temp.score < 17]

        if cards_dealt <= 2:
            # print(final)
            # print([temp for temp in final if temp.score == 21])
            probability_of_blackjack = sum([temp.chance for temp in final if temp.score == 21])
    
        
    
    # print(final)
    scores = np.array([[f.score, f.chance] for f in final])
    valcounts = pd.DataFrame(scores).groupby(0).sum()
    valcounts = pd.Series(index=valcounts.index.values.astype(int), data=valcounts.values.flatten())
    valcounts[0] = valcounts[valcounts.index > 21].sum()
    valcounts = valcounts[valcounts.index <= 21].sort_index(ascending=True)
    valcounts[21] -= probability_of_blackjack
    valcounts[22] = probability_of_blackjack

    # TO NORM ALL EXCEPT DEALER BLACKJACK, USE:
    # valcounts[:21] /= (1 - probability_of_blackjack)

    print(f"\n{valcounts}")  
    print("SUM", valcounts.sum())
    return valcounts



# valcounts2 = get_dealer_score_probabilities(deck, dealer=dealer)

cards in deck 310

0     0.244856
17    0.150225
18    0.150971
19    0.151401
20    0.151063
21    0.151484
22    0.000000
dtype: float64
SUM 1.0


In [123]:
# res = pd.concat((valcounts, valcounts2), axis=1, keys=["new", "old"])
# res["diff"] = res["new"] - res["old"]
# res

,new,old,diff
0,0.244856,0.244856,-4.440892e-16
17,0.150225,0.150225,6.383782e-16
18,0.150971,0.150971,4.440892e-16
19,0.151401,0.151401,3.885781e-16
20,0.151063,0.151063,-1.387779e-16
21,0.151484,0.151484,2.775558e-17
22,0.000000,0.000000,0.000000e+00
